In [2]:
import requests
import json
import os
from pathlib import Path

In [3]:
def read_config_file(file_path):
    config = {
        'folder_id': None,
        'api_key': None,
        'I_AM_TOKEN': None
    }
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue

                if '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip().strip("'").strip('"')

                    if key in config:
                        config[key] = value

    except FileNotFoundError:
        raise FileNotFoundError(f"Файл конфигурации не найден: {file_path}")
    except Exception as e:
        raise Exception(f"Ошибка при чтении файла конфигурации: {str(e)}")

    for key, value in config.items():
        if value is None:
            raise ValueError(f"Пропущен один из объектов конфигурации: {key}")

    return config


In [4]:
notebook_dir = Path().resolve()
config_path  = notebook_dir / "config.json"
with open(config_path, "r", encoding="utf-8") as f:
    cfg = json.load(f)
BASE_DIR = Path(cfg["base_directory"])
path = BASE_DIR / "keys.txt"
config = read_config_file(path)
folder_id  = config['folder_id']
api_key = config['api_key']
I_AM_TOKEN = config['I_AM_TOKEN']

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

headers = {
    "Authorization": f"Bearer {I_AM_TOKEN}",
}

In [5]:
def extract_response_text(response_json):
    alternatives = response_json.get('result', {}).get('alternatives', [])
    if alternatives:
        message = alternatives[0].get('message', {})
        text = message.get('text', '')
        return text
    return "Нет ответа от модели"


def gpt_request(prompt):
    data = {
        "modelUri": f"gpt://{folder_id}/yandexgpt/rc",
        "completionOptions": {
            "temperature": 0.3
        },
        "messages": [
            {
                "role": "user",
                "text": prompt
            }
        ],
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        response_json = response.json()
        generated_text = extract_response_text(response_json)
    else:
        print("Ошибка:", response.status_code, response.text)
    return generated_text

In [ ]:
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def process_text_with_gpt(text):
    prompt = """
     Ты — эксперт в области обработки текстов и создания структурированных конспектов. Твоя задача — преобразовать распознанный текст из видео-лекции в учебный конспект с четкой структурой и markdown-разметкой. Инструкции по выполнению задачи:

    Очисти текст:

    Убери лишние слова, разговорные выражения, повторы и неуместные фразы.

    Расставь знаки препинания, исправь ошибки в терминах и их оформлении.

    Сделай текст строгим и академичным.

    Структурируй текст:

    Раздели текст на смысловые блоки.

    Начни с определения области применения, затем классифицируй задачи, перейди к инструментам и их особенностям.

    Четко раздели общие возможности и ограничения.

    Добавь markdown-разметку, ответ должен быть именно в виде markdown-кода:

    Используй заголовки (#, ##, ###) для разделов и подразделов.

    Оформи списки (-, * или 1.) для перечисления.

    Выдели термины и ключевые моменты с помощью жирного текста или курсива.

    Можешь добавлять другие виды markdown разметки, которую посчитаешь нужным, например таблицы и тому подобное

    Проверь результат:

    Убедись, что текст логичен, структурирован и удобен для изучения.

    Примеры инструментов должны быть описаны кратко и по делу.

    Важно:

    Текст должен быть четким, логичным и готовым для использования в учебных целях.

    Проверь пунктуацию, термины и оформление.

    Важно: сохранение специальных меток

    В тексте встречаются специальные метки вида:
    ⟪000239s_top_7.jpg⟫
    ⟪XXXXXXs_top_YYY.jpg⟫

    Это крайне важные метки, которые служат для вставки изображений.
    Пожалуйста, НЕ изменяйте, НЕ удаляйте и НЕ переставляйте местами эти метки.
    Любой текст, заключенный в символы ⟪ и ⟫, надо сохранить в неизменном виде.
    Даже если вы сокращаете, перефразируете или обобщаете текст, такие метки
    обязаны оставаться без изменения в том же месте, где они были.

    Каждую метку расположи в месте, максимально
    подходящему по контексту из оригинального текста. Нельзя вставлять несколько меток
    в одно место или в конец текста.

    Вот текст, который нужно обработать:
    ```{}```
    """

    response = gpt_request(prompt.format(text))

    return response

In [ ]:
def clear_simple_text(text):
    prompt = """
    Ты — профессиональный редактор. Перед тобой текст, распознанный из видео (ASR-расшифровка). Он не содержит знаков препинания и имеет грамматические ошибки.

    Задача:
    Преобразуй этот текст в грамотный, читаемый вариант без изменения смысла, добавления или удаления слов.

    Правила обработки:

    Пунктуация:

    Расставь точки, запятые, тире, двоеточия и другие знаки препинания

    Восстанови вопросительные и восклицательные предложения (если это очевидно из контекста)

    Оформи прямую речь (если есть)

    Грамматика:

    Исправь слитное написание слов ("также" вместо "так же" где нужно)

    Поправь очевидные падежные/родовые ошибки ("в течение дня" вместо "в течении дня")

    Восстанови правильный порядок слов, если это не меняет смысл

    Форматирование:

    Раздели текст на абзацы по смыслу

    Сохрани все слова и фразы из оригинала

    Не добавляй новые слова и не удаляй существующие

    Важно:

    Сохраняй разговорный стиль, если он был в оригинале

    Не делай текст более "литературным" — только грамматическая коррекция

    Сомневаешься в пунктуации — выбирай самый простой вариант

    В тексте встречаются специальные метки вида:
    ⟪000239s_top_7.jpg⟫
    ⟪XXXXXXs_top_YYY.jpg⟫

    Это крайне важные метки, которые служат для вставки изображений.
    Пожалуйста, НЕ изменяйте, НЕ удаляйте и НЕ переставляйте местами эти метки.
    Любой текст, заключенный в символы ⟪ и ⟫, надо сохранить в неизменном виде.
    Даже если вы сокращаете, перефразируете или обобщаете текст, такие метки
    обязаны оставаться без изменения в том же месте, где они были.

    Каждую метку расположи в месте, максимально
    подходящему по контексту из оригинального текста. Нельзя вставлять несколько меток
    в одно место или в конец текста.

    Вот текст, который нужно обработать:
    ```{}```
    """

    response = gpt_request(prompt.format(text))

    return response

In [ ]:
%pip install openai

In [9]:
from openai import OpenAI

client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=api_key,
)


def deepseek_request(prompt):
    completion = client.chat.completions.create(
        #model="deepseek-ai/DeepSeek-V3",
        model = "deepseek-ai/DeepSeek-V3",
        messages=[
        {
            "role": "user",
            "content": prompt
        }],
        temperature=0.6
    )
    result = completion.choices[0].message.content
    return result

In [ ]:
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def mermaid_text_with_deepseek(text):
    prompt = """
      Ты — профессиональный редактор учебных материалов с навыками визуализации данных.
      Твоя задача — преобразовать сырой текст лекции в структурированный markdown-конспект с интерактивными диаграммами mermaid.js, ответ верни в виде готового markdown конспекта с вставленными схемами.

      Инструкции:

      Анализ контента:

      Выяви в тексте:

      Классификации (типы/категории/виды)
      Иерархические структуры
      Процессы и последовательности
      Сравнительные таблицы в зачаточной форме
      Взаимосвязи между концепциями

      Генерация диаграмм:

      Для найденных структур создай код mermaid.js:

      Flowchart — для процессов и алгоритмов
      ClassDiagram — для классификаций и отношений
      Graph — для связей между концепциями
      Timeline — для хронологических последовательностей

      Интеграция в текст:

      Размещай диаграммы:

      После первого упоминания ключевой структуры
      В разделах "Классификация" или "Архитектура"
      Перед/после сравнительных таблиц
      Добавляй пояснение к диаграмме (1-2 предложения)

      Контроль качества:

      Проверь синтаксис mermaid через официальный редактор,
      не используй кавычки внутри диаграм. Это синтаксически неверно.

      Убедись, что диаграмма:

      Не дублирует текст, а дополняет его
      Соответствует уровню сложности материала
      Имеет логичную направленность (слева-направо/сверху-вниз)
      Содержит только релевантные элементы

      Важно: сохранение специальных меток

      В тексте встречаются специальные метки вида:
      ⟪000239s_top_7.jpg⟫
      ⟪XXXXXXs_top_YYY.jpg⟫

      Это крайне важные метки, которые служат для вставки изображений.
      Пожалуйста, НЕ изменяйте, НЕ удаляйте и НЕ переставляйте местами эти метки.
      Любой текст, заключенный в символы ⟪ и ⟫, надо сохранить в неизменном виде.
      Даже если вы сокращаете, перефразируете или обобщаете текст, такие метки
      обязаны оставаться без изменения в том же месте, где они были.

      Каждую метку расположи в месте, максимально
      подходящему по контексту из оригинального текста. Нельзя вставлять несколько меток
      в одно место или в конец текста.

      Текст для обработки:
      ```{}```
    """

    response = deepseek_request(prompt.format(text))

    return response

In [ ]:
def replace_image_markers(final_output_path, base_name, txt_file, final_output_name):
    """
    Открывает файл по пути final_output_path, заменяет все метки вида
      ⟪XXXs_top_YYY.jpg⟫ на Markdown-ссылки вида
      ![](images/{base_name}/XXXs_top_YYY.jpg)
    и сохраняет изменения обратно в файл.

    Аргументы:
      final_output_path (str): путь к Markdown файлу.
      base_name (str): имя видео (без расширения), используется в пути к изображениям.
      txt_file (str): исходное имя TXT файла (для вывода сообщения).
      final_output_name (str): итоговое имя Markdown файла (для вывода сообщения).
    """
    # Чтение содержимого файла
    with open(final_output_path, 'r', encoding='utf-8') as f:
        md_content = f.read()

    # Регулярное выражение для поиска меток с именами файлов .jpg
    pattern = r"⟪([\w\d_]+\.jpg)⟫"
    # Формирование замены: вставляем путь images/{base_name}/имя_файла
    replacement = f"![](images/{base_name}/\\1)"
    updated_md_content = re.sub(pattern, replacement, md_content)

    # Запись обновленного содержимого обратно в файл
    with open(final_output_path, 'w', encoding='utf-8') as f:
        f.write(updated_md_content)

In [12]:
def process_and_generate_conspect(input_folder, output_folder, pretty_text_directory):
    """
    Для каждого txt файла с распознанной речью:
      - Формирует markdown конспект из текста,
      - Генерирует и вставляет mermaid схемы в конспект.
    """
    txt_files = [f for f in os.listdir(input_folder)
                if f.endswith('.txt')]

    for txt_file in txt_files:
        base_name = os.path.splitext(txt_file)[0]
        final_output_name = f"{base_name}_conspect.md"
        final_output_path = os.path.join(output_folder, final_output_name)

        final_pretty_text_name = f"{base_name}_text.txt"
        final_pretty_text_path = os.path.join(pretty_text_directory, final_pretty_text_name)

        if os.path.exists(final_output_path):
            print(f"Файл {final_output_name} уже существует. Пропускаем")
            continue

        try:
            # Обработка GPT
            input_path = os.path.join(input_folder, txt_file)
            text = read_text_from_file(input_path)

            simple_text = clear_simple_text(text)

            processed_text = process_text_with_gpt(text)

            # Генерация Mermaid
            mermaid_text = mermaid_text_with_deepseek(processed_text)
            mermaid_text = mermaid_text.removeprefix("```markdown\n").removesuffix("\n```")

            with open(final_output_path, 'w', encoding='utf-8') as f:
                f.write(mermaid_text)

            with open(final_pretty_text_path, 'w', encoding='utf-8') as f:
                f.write(simple_text)

            replace_image_markers(final_output_path, base_name, txt_file, final_output_name)
            replace_image_markers(final_pretty_text_path, base_name, txt_file, final_output_name)

            print(f"Успешно обработан: {txt_file} -> {final_output_name}")

        except Exception as e:
            print(f"Ошибка при обработке файла {txt_file}: {str(e)}")

In [13]:
import os
import re
import json

In [14]:
def insert_frame_images(transcription, frames_dir):
    """
    Для каждого файла кадра в папке frames_dir с именем вида "149s_top_6.jpg":
      - извлекает временную метку (например, 149),
      - определяет соответствующий сегмент транскрипции,
      - вставляет маркер изображения в формате "⟪149s_top_6.jpg⟫" после последней точки предложения.
    """
    print(f"[DEBUG] Обработка изображений из папки: {frames_dir}")
    # Получаем список файлов изображений (поддерживаются форматы jpg и png)
    frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(('.jpg', '.png'))]
    print(f"[DEBUG] Найдено {len(frame_files)} файлов изображений: {frame_files}")

    # Создаем словарь: время кадра (float) -> имя файла
    frame_mapping = {}
    for filename in frame_files:
        match = re.match(r'(\d+)s', filename)
        if match:
            timestamp = float(match.group(1))
            frame_mapping[timestamp] = filename
            print(f"[DEBUG] Файл {filename}: timestamp {timestamp}")
        else:
            print(f"[DEBUG] Файл {filename} не соответствует ожидаемому формату")

    # Обрабатываем каждый кадр: ищем соответствующий сегмент транскрипции
    for frame_time, image_file in frame_mapping.items():
        segment_found = None
        for seg in transcription.get("segments", []):
            if seg["start"] <= frame_time <= seg["end"]:
                segment_found = seg
                print(f"[DEBUG] Timestamp {frame_time} найден в сегменте {seg.get('id', 'N/A')} (диапазон {seg['start']} - {seg['end']})")
                break
        if not segment_found and transcription.get("segments", []):
            segment_found = min(transcription["segments"], key=lambda seg: abs(seg["start"] - frame_time))
            print(f"[DEBUG] Timestamp {frame_time} не попадает ни в один сегмент, выбран ближайший сегмент {segment_found.get('id', 'N/A')} (диапазон {segment_found['start']} - {segment_found['end']})")
        if segment_found:
            marker = f" ⟪{image_file}⟫"
            seg_text = segment_found.get("text", "")
            last_dot = seg_text.rfind('.')
            if last_dot != -1:
                new_text = seg_text[:last_dot+1] + marker + seg_text[last_dot+1:]
            else:
                new_text = seg_text + marker
            print(f"[DEBUG] Вставка маркера {marker} в сегмент {segment_found.get('id', 'N/A')}")
            segment_found["text"] = new_text
        else:
            print(f"[DEBUG] Не найден сегмент для файла {image_file} с timestamp {frame_time}")
    return transcription


def process_frame_markers(video_directory, whisper_json_directory, whisper_text_directory, extracted_frames_base):
    """
    Функция вставки меток:
      1. Для каждого видео в video_directory проверяется наличие JSON транскрипции и соответствующего TXT.
      2. Поиск папки с извлечёнными кадрами (варианты имени – оригинальное, с заменой пробелов и наоборот).
      3. Если папка найдена, загружается JSON, выполняется вставка маркеров через insert_frame_images,
         итоговый текст (объединение сегментов) формируется заново и сохраняется в TXT.
    """
    for video_file in os.listdir(video_directory):
        if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        video_name_no_ext = os.path.splitext(video_file)[0]
        video_name_clean = video_name_no_ext.replace(" ", "_")
        json_output_path = os.path.join(whisper_json_directory, video_name_clean + ".json")
        txt_output_path = os.path.join(whisper_text_directory, video_name_clean + ".txt")

        # Если JSON или TXT не существует, пропускаем видео (так как транскрипция должна быть выполнена первой)
        if not os.path.exists(json_output_path):
            print(f"[INFO] JSON транскрипция для видео '{video_name_no_ext}' не найдена. Пропускаем вставку маркеров.")
            continue
        if not os.path.exists(txt_output_path):
            print(f"[INFO] TXT файл для видео '{video_name_no_ext}' не найден. Пропускаем вставку маркеров.")
            continue

        # Загружаем JSON транскрипции
        with open(json_output_path, "r", encoding="utf-8") as f:
            transcription = json.load(f)

        # Поиск папки с извлечёнными кадрами: варианты имени – оригинальное, с заменой пробелов и наоборот
        folder_options = {
            video_name_no_ext,
            video_name_no_ext.replace(" ", "_"),
            video_name_no_ext.replace("_", " ")
        }
        print(f"[DEBUG] Варианты имени папки для кадров: {folder_options}")
        frames_dir = None
        for folder in folder_options:
            potential_dir = os.path.join(extracted_frames_base, folder)
            print(f"[DEBUG] Проверка папки: {potential_dir}")
            if os.path.exists(potential_dir):
                frames_dir = potential_dir
                print(f"[INFO] Папка с кадрами найдена: {frames_dir}")
                break

        if frames_dir:
            # Выполняем вставку маркеров в транскрипцию
            transcription = insert_frame_images(transcription, frames_dir)
            # Формируем новый итоговый текст (объединение сегментов с маркерами)
            if "segments" in transcription:
                transcription["text"] = " ".join(seg["text"] for seg in transcription["segments"])
                print(f"[DEBUG] Итоговый текст с метками сформирован. Длина текста: {len(transcription['text'])} символов")
            else:
                print(f"[WARN] Нет сегментов в транскрипции для видео '{video_name_no_ext}' после вставки меток")

            # Сохраняем обновлённый TXT файл (при этом JSON можно оставить без изменений или обновить по необходимости)
            with open(txt_output_path, "w", encoding="utf-8") as f:
                f.write(transcription.get("text", ""))
            print(f"[INFO] TXT файл обновлён с метками: {txt_output_path}")
        else:
            print(f"[INFO] Папка с кадрами не найдена для видео '{video_name_no_ext}'")


In [15]:
# video_directory = get_relative_path('local', 'course', 'input')
# whisper_json_directory = get_relative_path('local', 'course', 'work', 'whisper_json')
# whisper_text_directory = get_relative_path('local', 'course', 'work', 'whisper_text')
# extracted_frames_base = get_relative_path('local', 'course', 'output', 'images')
# output_folder = get_relative_path('local', 'course', 'output')
# pretty_text_directory = get_relative_path('local', 'course', 'work', 'pretty_text')

# Каталоги для транскрипции на основе BASE_DIR из config.json
video_directory         = BASE_DIR / "input"
whisper_json_directory  = BASE_DIR / "work"  / "whisper_json"
whisper_text_directory  = BASE_DIR / "work"  / "whisper_text"
extracted_frames_base   = BASE_DIR / "output"/ "images"
output_folder   = BASE_DIR / "output"
pretty_text_directory   = BASE_DIR / "work"  / "pretty_text"

# Создаем все необходимые директории
os.makedirs(video_directory, exist_ok=True)
os.makedirs(whisper_json_directory, exist_ok=True)
os.makedirs(whisper_text_directory, exist_ok=True)
os.makedirs(extracted_frames_base, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
os.makedirs(pretty_text_directory, exist_ok=True)

In [ ]:
# Этап 1: Вставка меток изображений в транскрипцию и обновление TXT-файла
process_frame_markers(video_directory, whisper_json_directory, whisper_text_directory, extracted_frames_base)

In [ ]:
# Этап 2: Составление конспекта из TXT и добавление mermaid схем
process_and_generate_conspect(whisper_text_directory, output_folder, pretty_text_directory)